In [ ]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import geopandas as gpd

# Boston Crime

In [ ]:
m_1 = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=10)
m_1

In [ ]:
# Load the data
crimes = pd.read_csv("data/crimes-in-boston/crimes-in-boston/crime.csv", encoding='latin-1')

# Drop rows with missing locations
crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# Focus on major crimes in 2018
crimes = crimes[crimes.OFFENSE_CODE_GROUP.isin([
    'Larceny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary', 
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment', 
    'Manslaughter'])]
crimes = crimes[crimes.YEAR>=2018]

# Print the first five rows of the table
crimes.head()


In [ ]:
roberies = crimes.query("OFFENSE_CODE_GROUP == 'Robbery'")
roberies.head()

In [ ]:
fmap = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)
mc = MarkerCluster()
for idx, row in roberies.iterrows():
    mc.add_child(Marker([row['Lat'], row['Long']]))
fmap.add_child(mc)

In [ ]:
def set_color(val):
    if val < 12:
        return "forestgreen"
    else:
        return "darkred"

In [ ]:
fmap = folium.Map(location=[42.32, -71.0589], tiles="cartodbpositron", zoom_start=13)
for ix, row in roberies.iterrows():
    Circle(location=[row['Lat'], row['Long']], radius=20, color=set_color(row['HOUR'])).add_to(fmap)
fmap

In [ ]:
fmap = folium.Map(location=[42.32, -71.0589], tiles="cartodbpositron", zoom_start=13)
HeatMap(data=crimes[['Lat', 'Long']], radius=10).add_to(fmap)
fmap

In [ ]:
districts = gpd.read_file("data/Police_Districts/Police_Districts/Police_Districts.shp")
districts = districts[["DISTRICT", "geometry"]].set_index("DISTRICT")
districts.head()

In [ ]:
crimes.DISTRICT.value_counts()

In [ ]:
fmap = folium.Map(location=[42.32, -71.0589], tiles="cartodbpositron", zoom_start=11)
fmap.add_child(
    Choropleth(
        geo_data=districts.__geo_interface__,
        data = crimes.DISTRICT.value_counts(),
        key_on="feature.id",
        ledgend_name="Places where they did that crime"
    
    )
)

# Japan Earthquakes

In [ ]:
plates = gpd.read_file("data/Plate_Boundaries/Plate_Boundaries/Plate_Boundaries.shp")
plates.head()

In [ ]:
earthquakes = pd.read_csv("data/earthquakes1970-2014.csv")
earthquakes.head()

In [ ]:
plates["coordinates"] = plates.apply(lambda x: [(b, a) for (a, b) in list(x.geometry.coords)], axis="columns")

In [ ]:
m = folium.Map(location=[35, 136], titles="cartodbpositron", zoom_start=5)
for i in range(len(plates)):
    m.add_child(folium.PolyLine(locations=plates.coordinates.iloc[i], color='black', weight=2))
m.add_child(HeatMap(
    data=earthquakes[["Latitude", "Longitude"]], radius=10
))

In [ ]:
def set_color(val):
    if val < 50:
        return 'forestgreen'
    elif val < 100:
        return 'darkorange'
    else:
        return 'darkred'

In [ ]:
m = folium.Map(location=[35, 136], tiles="cartodbpositron", zoom_start=5)
for i in range(len(plates)):
    m.add_child(folium.PolyLine(locations=plates.coordinates.iloc[i], color='black', weight=2))
for i in range(len(earthquakes)):
    m.add_child(folium.Circle(location=[earthquakes.iloc[i]["Latitude"], earthquakes.iloc[i]["Longitude"]], radius=2000, color=set_color(earthquakes.iloc[i]["Depth"])))

In [ ]:
m

In [ ]:
prefectures = gpd.read_file("data/japan-prefecture-boundaries/japan-prefecture-boundaries/japan-prefecture-boundaries.shp")
prefectures.set_index("prefecture", inplace=True)
prefectures.head()

In [ ]:
population = pd.read_csv("data/japan-prefecture-population.csv")
population.set_index("prefecture", inplace=True)
population.head()

In [ ]:
area_sqkm = pd.Series(prefectures.geometry.to_crs(epsg=32654).area / 10**6, name="area_sqkm")
stats = population.join(area_sqkm)
stats["density"] = stats["population"] / stats["area_sqkm"]
stats.head()

In [ ]:
m = folium.Map(location=[35, 136], tiles="cartodbpositron", zoom_start=5)

In [ ]:
def set_color(value):
    if value > 6.5:
        return "darkred"
    else:
        return "forestgreen"

In [ ]:
m.add_child(Choropleth(
    geo_data=prefectures.geometry.__geo_interface__,
    data=stats["density"],
    key_on="feature.id"
))
for i in range(len(earthquakes)):
    m.add_child(folium.Circle(location=[earthquakes.iloc[i]["Latitude"], earthquakes.iloc[i]["Longitude"]], radius=earthquakes.iloc[i]["Magnitude"]**5.5, color=set_color(earthquakes.iloc[i]["Magnitude"])))

In [ ]:
m